In [1]:
import datetime
import pickle
import logging
import csv
import os
from enum import Enum
import string


logging.basicConfig(
    format="%(levelname)s %(asctime)s [%(filename)s:%(lineno)d]: %(message)s",
    level=logging.DEBUG,
)

import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.tokenize import sent_tokenize
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
nltk.download('punkt_tab')
nltk.download('stopwords')

stop_words = set(stopwords.words('russian'))
stemmer = SnowballStemmer("russian")

[nltk_data] Downloading package punkt_tab to /home/almaz/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /home/almaz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
model = LogisticRegression(
        max_iter=1900  # 1900
    )
vectorizer = TfidfVectorizer(stop_words=list(stop_words))

In [4]:
allowed_symbols = (
        string.ascii_lowercase +
        " -+%.," +
        # string.digits +
        "абвгдежзийклмнопрстуфхцчшщъыьэюя"
)
def getClearText(text: str) -> str:
    cleared_text = ""
    for letter in text:
        if letter in allowed_symbols:
            cleared_text += letter

    return cleared_text

replace_dict = {
    " +": " плюс ",
    " -": " минус ",
    "%": " процент "
}
def replaceSymbols(text: str) -> str:
    new_text = text
    for key, value in replace_dict.items():
        new_text = new_text.replace(key, value)
    return new_text

def getFullClearText(text: str) -> str:
    text = text.lower()
    cleared_text = getClearText(text)
    cleared_text = replaceSymbols(cleared_text)
    return cleared_text

In [32]:
def fit(db_path: str):
    global model, vectorizer

    df = pd.read_csv(db_path)
    df_texts = df.Text
    print(df.head(200))
    values = np.array([-1 if value < -0.08 else 1 for value in df.Value])

    cleared = []
    for text in df_texts:
        cleared.append(getFullClearText(text))

    vectorized = vectorizer.fit_transform(cleared)
    model.fit(vectorized, values)

In [8]:
def predict(ticker: str) -> str:
    csv_path = f"../parse_experiments/recent_news/{ticker}.csv"
    try:
        with open(
                csv_path,
                mode='r',
                newline='',
                encoding='utf-8'
        ) as file:
            file.seek(0)
            reader = csv.DictReader(file)
            news = []
            predictors = []

            for i, row in enumerate(reader):
                title = row['Title']
                text = row['Text']
                time = row['Time']
                url = row['Url']

                time = datetime.datetime.strptime(time, "%d.%m.%Y, %H:%M")
                if time < datetime.datetime.now() - datetime.timedelta(days=365 * 2):
                    continue

                news.append({})
                news[i]["text"] = row["Text"]
                news[i]["url"] = row["Url"]
                news[i]["title"] = row["Title"]
                news[i]["text_cleared"] = getFullClearText(text)

                predictors.append(news[i]["text_cleared"])

            if len(news) == 0:
                return f"Не нашлось новостей за последние два года для {ticker} :("

            vectorized = vectorizer.transform(predictors)

            y_predict = model.predict(vectorized)

            logging.info(f"PREDICT {y_predict}")

            verdict = f"Оценка новостей связанных с компанией {ticker.upper()}:\n"

            for i in range(len(news)):
                match y_predict[i]:
                    case -1:
                        mark = "💩"
                    case 1:
                        mark = "🤑"
                    case other:
                        mark = f"{other}: непредвиденный аргумент y_predict[i]"
                sample = f"{mark} {i + 1}. [{news[i]['title']}]({news[i]['url']})"
                verdict += sample + "\n"

            overall_mark = "НЕПЕРСПЕКТИВНАЯ" if sum(y_predict) < 0 else "ПЕРСПЕКТИВНАЯ"
            verdict += ("\n"
                        f"Общая оценка: *{overall_mark}*")

            return verdict

    except FileNotFoundError as ex:
        return f"Не нашлось новостей за последние два года для {ticker} :("



In [33]:
fit("database_some.csv")

                                                 Title  \
0    Рынок акций РФ закрылся падением, Индекс МосБи...   
1    Рынок акций РФ закрылся падением, Индекс МосБи...   
2    Рынок акций РФ закрылся падением, Индекс МосБи...   
3    Рынок акций РФ закрылся ростом, Индекс МосБирж...   
4    Рынок акций РФ закрылся ростом, Индекс МосБирж...   
..                                                 ...   
195  Рынок акций Московской биржи по состоянию на 1...   
196  Акционеры "Лензолота" одобрили дивиденды за 9 ...   
197  BNY откажется от функций депозитария по распис...   
198  Рынок акций Московской биржи по состоянию на 1...   
199  Рынок акций Московской биржи по состоянию на 1...   

                                                  Text               Time  \
0    © Reuters. Рынок акций РФ закрылся падением, И...  21.06.2018, 19:35   
1    © Reuters. Рынок акций РФ закрылся падением, И...  15.05.2018, 19:35   
2    Рынок акций РФ закрылся падением, Индекс МосБи...  08.06.2018, 19:3

In [29]:
predict("T")

INFO 2025-01-15 00:10:42,480 [332346506.py:40]: PREDICT [ 1 -1  1  1  1  1  1  1 -1 -1]


'Оценка новостей связанных с компанией T:\n🤑 1. [Рынок акций РФ закрылся ростом, Индекс МосБиржи (ММВБ) прибавил 3,33%](https://ru.investing.com/news/stock-market-news/article-2610900)\n💩 2. [Т-банк хочет открыть два филиала в Китае](https://ru.investing.com/news/general/article-2609825)\n🤑 3. [Рынок акций РФ закрылся ростом, Индекс МосБиржи (ММВБ) прибавил 1,50%](https://ru.investing.com/news/stock-market-news/article-2609098)\n🤑 4. [Рынок акций РФ закрылся ростом, Индекс МосБиржи (ММВБ) прибавил 9,19%](https://ru.investing.com/news/stock-market-news/article-2607248)\n🤑 5. [Рынок акций РФ закрылся ростом, Индекс МосБиржи (ММВБ) прибавил 1,15%](https://ru.investing.com/news/stock-market-news/article-2604494)\n🤑 6. [Рынок акций РФ закрылся падением, Индекс МосБиржи (ММВБ) снизился на 0,88%](https://ru.investing.com/news/stock-market-news/article-2599618)\n🤑 7. [Рынок акций РФ закрылся падением, Индекс МосБиржи (ММВБ) снизился на 1,94%](https://ru.investing.com/news/stock-market-news/art